## ELFCAR / iso-orbital indicator ($\alpha$) analysis

Calculate the alpha value from the ELFCAR, and write to a format that can be read by VESTA

---
## Header

#### Global variables

In [1]:
PROJECT = 'SCAN project'

#### Imports

In [2]:
import os
from pathlib import Path
import re
import numpy as np
#import xlrd
from matplotlib import pyplot as plt
import plotly.graph_objects as go

In [3]:
from pymatgen.ext.matproj import MPRester, MPRestError
from monty.serialization import loadfn, dumpfn
from pymatgen import Structure, Composition
from pymatgen.analysis.reaction_calculator import ComputedEntry, ComputedReaction
from pymatgen.util.plotting import pretty_plot, periodic_table_heatmap
from pymatgen.core import periodic_table
from pymatgen.io.vasp.outputs import Elfcar, Chgcar, Poscar, VolumetricData

from scipy.stats import linregress
from scipy.interpolate import RegularGridInterpolator
#from adjustText import adjust_text
from sklearn.metrics import max_error, mean_absolute_error, mean_squared_error

#### Settings and utility functions

In [4]:
%load_ext autoreload
%autoreload 2

#### Set Working Directory

In [5]:
workdir = Path(re.sub("(?<={})[\w\W]*".format(PROJECT), "", str(Path.cwd())))
os.chdir(workdir)

data_dir = workdir / 'SCAN_elfcar_v1'
#pipeline_dir = workdir / 'analysis' / 'pipeline'
output_dir = workdir / 'vasp_files'

In [6]:
print(workdir)
print(data_dir)
#print(pipeline_dir)
print(output_dir)

/mnt/c/Users/Ayush/Desktop/elf-analysis
/mnt/c/Users/Ayush/Desktop/elf-analysis/SCAN_elfcar_v1
/mnt/c/Users/Ayush/Desktop/elf-analysis/vasp_files


---
## Main Code

### Create a simple dict containing formula: pymatgen ELFCAR object and write files that VESTA can read

In [7]:
def make_elfcar_dict(): 
    elfcar_dict = {}
    #target_dir = output_dir / 'iso-orbital-alpha'
    elf_target_dir = output_dir / 'elf'
    print('Destination:', output_dir)
    print('Compounds:')
    count = 0
    # loop through the directories containing the calcs
    block_dir = data_dir
    for d1 in block_dir.iterdir():
        # loop through the "launcher" dirs in each "block"
        if '.DS_Store' in str(d1):
            continue
        for d2 in d1.iterdir():
            count+=1
            if '.DS_Store' in str(d2):
                continue
                
            poscar = d2 / 'POSCAR.relax2.gz'
            elfcar = d2 / 'ELFCAR.relax2.gz'
                
            try:
                def file_helper():
                    nonlocal elfcar
                    formula = Structure.from_file(poscar).composition.reduced_formula
                    elfcar = Elfcar.from_file(elfcar)
                    if formula in elfcar_dict:
                        print('Duplicate formula {} found'.format(formula))
                    elfcar_dict.update({formula:elfcar})
                    # Write a file for reading in VESTA
                    elfcar.write_file(Path(elf_target_dir,formula+'.vasp'))
                    #chgcar.write_file(Path(chg_target_dir,formula+'.vasp'))
                    print(formula)
                file_helper()
            except FileNotFoundError as e:
                poscar = d2 / 'POSCAR.relax3.gz'
                elfcar = d2 / 'ELFCAR.relax3.gz'
                file_helper()
            except Exception as exc:
                print({exc})
                continue
        
    print('Traversed {} directories'.format(count))
    return elfcar_dict

elfcar_dict = make_elfcar_dict()
#elfcar_dict = generate_file(os.path.join('3_data analysis','2_pipeline','elfcar_dict.json'),make_elfcar_dict)

Destination: /mnt/c/Users/Ayush/Desktop/elf-analysis/vasp_files
Compounds:
TiC
IrO2
TaN
TiO2
MgSO4
MnBr2
Mn3O4
Pt5Se4
K2Si4O9
Al2S3
Fe2O3
Eu2O3
Traversed 16 directories


In [8]:
elfcar_dict

{'TiC': <pymatgen.io.vasp.outputs.Elfcar at 0x7f1e52a924f0>,
 'IrO2': <pymatgen.io.vasp.outputs.Elfcar at 0x7f1e52a924c0>,
 'TaN': <pymatgen.io.vasp.outputs.Elfcar at 0x7f1e52aa85e0>,
 'TiO2': <pymatgen.io.vasp.outputs.Elfcar at 0x7f1e52aa8490>,
 'MgSO4': <pymatgen.io.vasp.outputs.Elfcar at 0x7f1e52d27ee0>,
 'MnBr2': <pymatgen.io.vasp.outputs.Elfcar at 0x7f1e5296de80>,
 'Mn3O4': <pymatgen.io.vasp.outputs.Elfcar at 0x7f1e528e49d0>,
 'Pt5Se4': <pymatgen.io.vasp.outputs.Elfcar at 0x7f1e52a925b0>,
 'K2Si4O9': <pymatgen.io.vasp.outputs.Elfcar at 0x7f1e528e4dc0>,
 'Al2S3': <pymatgen.io.vasp.outputs.Elfcar at 0x7f1e52ab2d60>,
 'Fe2O3': <pymatgen.io.vasp.outputs.Elfcar at 0x7f1e52ab2e80>,
 'Eu2O3': <pymatgen.io.vasp.outputs.Elfcar at 0x7f1e528f6760>}

In [11]:
elfcar_dict['TiC'].as_dict()

NotImplementedError: Unable to automatically determine as_dict format from class. MSONAble requires all args to be present as either self.argname or self._argname, and kwargs to be present undera self.kwargs variable to automatically determine the dict format. Alternatively, you can implement both as_dict and from_dict.